# Oriented Bounding Boxes through Faster R-CNN

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import os
import sys
import glob
import cv2
import imageio.v2 as imageio
from IPython.display import display, Image

## Dataset Visualization

In [19]:
dataset_folder = "./dataset/files/"
img_dir = os.path.join(dataset_folder, "img")
annots_dir = os.path.join(dataset_folder, "annots")

In [20]:
images_path = glob.glob(os.path.join(img_dir, "*.jpg"))
annotations_path = [os.path.join(annots_dir, os.path.basename(img) + ".json") for img in images_path]

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(10, 10), constrained_layout=True)
for i, (img_path, annot_path) in enumerate(zip(images_path, annotations_path)):
    if i == 16:
        break
    img = plt.imread(img_path)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    with open(annot_path, "r") as f:
        annot = json.load(f)
        xc = [detec["obb"]["xc"] for detec in annot["objects"]]
        yc = [detec["obb"]["yc"] for detec in annot["objects"]]
        w = [detec["obb"]["w"] for detec in annot["objects"]]
        h = [detec["obb"]["h"] for detec in annot["objects"]]
        theta = [detec["obb"]["theta"] for detec in annot["objects"]]
        
        for x, y, width, height, angle in zip(xc, yc, w, h, theta):
            rect = cv2.boxPoints(((x, y), (width, height), angle))
            rect = np.intp(rect)
            cv2.drawContours(img_bgr, [rect], 0, (0, 255, 0), 4)
    
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    axs[i//4, i%4].imshow(img_rgb)
    axs[i//4, i%4].axis("off")

plt.suptitle("Dataset samples with annotations")
plt.show()


## Understanding Faster RCNN

### Changes made in code
- `dataset/st.py` - train-val-test split as 60-20-20
- added `val` as a parameter to the rpn class to visualize the validation set
- added separate loop for running on validation set in each epoch in `train.py`
- Images get resized in preprocessing before RPN so boxes are saved in 0,1 range

### Hyperparameters

- `rpn_post_nms_top_n_train` = 2000
- `rpn_fg_iou_thresh` = 0.7
-  `rpn_positive_fraction` = 0.5

In [22]:
def load_arrays(filename):
    """Loads multiple appended NumPy arrays from a file."""
    arrays = []
    with open(filename, "rb") as f:
        while True:
            try:
                arrays.append(np.load(f, allow_pickle=False))
            except EOFError:
                break 
    return arrays

In [23]:
epochs = 10
num_features = 5
num_images = 4
images_path = 'outputs/images/'

### Objectness Score

In [24]:
for hyperparam_idx in range(1,4):
    output_folder = f"outputs/{hyperparam_idx}/objectness"
    os.makedirs(f"{output_folder}/images", exist_ok=True)
    os.makedirs(f"{output_folder}/gifs", exist_ok=True)
    
    # read all the files
    features = []
    for i in range(num_features):
        features.append(load_arrays(f"{output_folder}/feature_{i}.npy"))
    for img_idx in range(num_images):
        for epoch in range(epochs):
            # make plots and save them
            fig, axs = plt.subplots(1, 5, figsize=(15, 3), constrained_layout=True)
            for i in range(5):
                img = features[i][num_images*epoch + img_idx]
                img = (img - img.min()) / (img.max() - img.min())
                axs[i].imshow(img)
                axs[i].axis("off")
                axs[i].set_title(f"Feature {i}")
            plt.suptitle(f"Hyperparams {hyperparam_idx} - Image {img_idx+1} - Epoch {epoch+1}")
            plt.savefig(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
            plt.close()
            
        # make gifs
        with imageio.get_writer(f"{output_folder}/gifs/img_{img_idx}.gif", mode='I') as writer:
            for epoch in range(epochs):
                writer.append_data(imageio.imread(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png"))
    
    # delete the images
    for img_idx in range(4):
        for epoch in range(epochs):
            os.remove(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
    os.rmdir(f"{output_folder}/images")
    

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2/objectness/feature_0.npy'

![](outputs/1/objectness/gifs/img_0.gif)
![](outputs/1/objectness/gifs/img_1.gif)
![](outputs/1/objectness/gifs/img_2.gif)
![](outputs/1/objectness/gifs/img_3.gif)
![](outputs/2/objectness/gifs/img_0.gif)
![](outputs/2/objectness/gifs/img_1.gif)
![](outputs/2/objectness/gifs/img_2.gif)
![](outputs/2/objectness/gifs/img_3.gif)
![](outputs/3/objectness/gifs/img_0.gif)
![](outputs/3/objectness/gifs/img_1.gif)
![](outputs/3/objectness/gifs/img_2.gif)
![](outputs/3/objectness/gifs/img_3.gif)
![](outputs/4/objectness/gifs/img_0.gif)
![](outputs/4/objectness/gifs/img_1.gif)
![](outputs/4/objectness/gifs/img_2.gif)
![](outputs/4/objectness/gifs/img_3.gif)

### Object Proposals

In [ ]:
for hyperparam_idx in range(1,4):
    output_folder = f"outputs/{hyperparam_idx}/object_proposals"
    output_file = f"{output_folder}/boxes.npy"
    os.makedirs(f"{output_folder}/images", exist_ok=True)
    os.makedirs(f"{output_folder}/gifs", exist_ok=True)
    
    list_boxes = load_arrays(output_file)

    for img_idx in range(num_images):
        for epoch in range(epochs):
            # make plots and save them
            fig, axs = plt.subplots(1,1, figsize=(8, 8), constrained_layout=True)
            img = plt.imread(images_path+"image_"+str(img_idx)+".jpg")
            width, height = img.shape[1], img.shape[0]
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            for box in list_boxes[epoch*num_images+img_idx]:
                box[0], box[1], box[2], box[3] = box[0] * width, box[1] * height, box[2] * width, box[3] * height
                box = [int(x) for x in box]
                rect = cv2.rectangle(img_bgr, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            axs.imshow(img_rgb)
            axs.axis("off")
            plt.suptitle(f"Hyperparams {hyperparam_idx} - Image {img_idx+1} - Epoch {epoch+1}")
            plt.savefig(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
            plt.close()
            
        # make gifs
        with imageio.get_writer(f"{output_folder}/gifs/img_{img_idx}.gif", mode='I') as writer:
            for epoch in range(epochs):
                writer.append_data(imageio.imread(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png"))
                
    # delete the images
    for img_idx in range(4):
        for epoch in range(epochs):
            os.remove(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
    os.rmdir(f"{output_folder}/images")


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2/object_proposals/boxes.npy'

| ![](outputs/1/object_proposals/gifs/img_0.gif) | ![](outputs/1/object_proposals/gifs/img_1.gif) |
|---------|---------|
| ![](outputs/1/object_proposals/gifs/img_2.gif) | ![](outputs/1/object_proposals/gifs/img_3.gif) |

| ![](outputs/2/object_proposals/gifs/img_0.gif) | ![](outputs/2/object_proposals/gifs/img_1.gif) |
|---------|---------|
| ![](outputs/2/object_proposals/gifs/img_2.gif) | ![](outputs/2/object_proposals/gifs/img_3.gif) |

| ![](outputs/3/object_proposals/gifs/img_0.gif) | ![](outputs/3/object_proposals/gifs/img_1.gif) |
|---------|---------|
| ![](outputs/3/object_proposals/gifs/img_2.gif) | ![](outputs/3/object_proposals/gifs/img_3.gif) |

| ![](outputs/4/object_proposals/gifs/img_0.gif) | ![](outputs/4/object_proposals/gifs/img_1.gif) |
|---------|---------|
| ![](outputs/4/object_proposals/gifs/img_2.gif) | ![](outputs/4/object_proposals/gifs/img_3.gif) |

### Bounding Box Assignments

In [ ]:
for hyperparam_idx in range(1,4):
    output_folder = f"outputs/{hyperparam_idx}/bb_assignments"
    background_file = f"{output_folder}/background_boxes.npy"
    foreground_file = f"{output_folder}/foreground_boxes.npy"
    ignore_file = f"{output_folder}/ignore_boxes.npy"
    os.makedirs(f"{output_folder}/images", exist_ok=True)
    
    background_boxes = load_arrays(background_file)
    foreground_boxes = load_arrays(foreground_file)
    ignore_boxes = load_arrays(ignore_file)
    
    for img_idx in range(num_images):
        bg_box = background_boxes[img_idx]
        fg_box = foreground_boxes[img_idx]
        ig_box = ignore_boxes[img_idx]
        # print(bg_box)
        
        # take boxes at equal intervals but total boxes should be 10
        bg_box = bg_box[::len(bg_box)//10]
        fg_box = fg_box[::len(fg_box)//10]
        ig_box = ig_box[::len(ig_box)//10]
        
        # make plots and save them
        fig, axs = plt.subplots(1,3, figsize=(15, 5), constrained_layout=True)
        img = plt.imread(images_path+"image_"+str(img_idx)+".jpg")
        width, height = img.shape[1], img.shape[0]
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img_bgr_bg = img_bgr.copy()
        img_bgr_fg = img_bgr.copy()
        img_bgr_ig = img_bgr.copy()
        
        for box in bg_box:
            box[0], box[1], box[2], box[3] = box[0] * width, box[1] * height, box[2] * width, box[3] * height
            box = [int(x) for x in box]
            rect = cv2.rectangle(img_bgr_bg, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
        img_rgb_bg = cv2.cvtColor(img_bgr_bg, cv2.COLOR_BGR2RGB)
        axs[0].imshow(img_rgb_bg)
        axs[0].axis("off")
        axs[0].set_title("Background")
        
        for box in fg_box:
            box[0], box[1], box[2], box[3] = box[0] * width, box[1] * height, box[2] * width, box[3] * height
            box = [int(x) for x in box]
            rect = cv2.rectangle(img_bgr_fg, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        img_rgb_fg = cv2.cvtColor(img_bgr_fg, cv2.COLOR_BGR2RGB)
        axs[1].imshow(img_rgb_fg)
        axs[1].axis("off")
        axs[1].set_title("Foreground")
        
        for box in ig_box:
            box[0], box[1], box[2], box[3] = box[0] * width, box[1] * height, box[2] * width, box[3] * height
            box = [int(x) for x in box]
            rect = cv2.rectangle(img_bgr_ig, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
        img_rgb_ig = cv2.cvtColor(img_bgr_ig, cv2.COLOR_BGR2RGB)
        axs[2].imshow(img_rgb_ig)
        axs[2].axis("off")
        plt.suptitle(f"Hyperparams {hyperparam_idx} - Image {img_idx+1}")
        plt.savefig(f"{output_folder}/images/img_{img_idx}.png")
        plt.close()
    break

![](outputs/1/bb_assignments/images/img_0.png)
![](outputs/1/bb_assignments/images/img_1.png)
![](outputs/1/bb_assignments/images/img_2.png)
![](outputs/1/bb_assignments/images/img_3.png)
![](outputs/2/bb_assignments/images/img_0.png)
![](outputs/2/bb_assignments/images/img_1.png)
![](outputs/2/bb_assignments/images/img_2.png)
![](outputs/2/bb_assignments/images/img_3.png)
![](outputs/3/bb_assignments/images/img_0.png)
![](outputs/3/bb_assignments/images/img_1.png)
![](outputs/3/bb_assignments/images/img_2.png)
![](outputs/3/bb_assignments/images/img_3.png)

### ROI Head Output

In [ ]:
for hyperparam_idx in range(1,4):
    output_folder = f"outputs/{hyperparam_idx}/roi_head_outputs"
    predictions_file = f"{output_folder}/predictions.npy"
    proposals_file = f"{output_folder}/proposals.npy"
    scores_file = f"{output_folder}/scores.npy"
    os.makedirs(f"{output_folder}/images", exist_ok=True)
    os.makedirs(f"{output_folder}/gifs", exist_ok=True)
    
    predictions = load_arrays(predictions_file)
    proposals = load_arrays(proposals_file)
    scores = load_arrays(scores_file)
    
    num_boxes = len(proposals[0])
    
    for img_idx in range(num_images):
        for epoch in range(epochs):
            # make plots and save them
            fig, axs = plt.subplots(1, 1, figsize=(8, 8), constrained_layout=True)
            img = plt.imread(images_path+"image_"+str(img_idx)+".jpg")
            width, height = img.shape[1], img.shape[0]
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # show proposal and prediction with score on top of each prediction
            for i in range(0,num_boxes,100):
                box = proposals[epoch*num_images+img_idx][i]
                box[0], box[1], box[2], box[3] = box[0] * width, box[1] * height, box[2] * width, box[3] * height
                box = [int(x) for x in box]
                rect = cv2.rectangle(img_bgr, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
                cv2.putText(img_bgr, f"{scores[epoch*num_images+img_idx][i][0]:.2f}", (box[0], box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                box_preds = predictions[epoch*num_images+img_idx][i]
                box_preds[0], box_preds[1], box_preds[2], box_preds[3] = box_preds[0] * width, box_preds[1] * height, box_preds[2] * width, box_preds[3] * height
                box_preds = [int(box[i] + box_preds[i]) for i in range(4)]
                rect = cv2.rectangle(img_bgr, (box_preds[0], box_preds[1]), (box_preds[2], box_preds[3]), (0, 255, 0), 2)
                
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            axs.imshow(img_rgb)
            axs.axis("off")
            plt.suptitle(f"Hyperparams {hyperparam_idx} - Image {img_idx+1} - Epoch {epoch+1}")
            plt.savefig(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
            plt.close()
            
        # make gifs
        with imageio.get_writer(f"{output_folder}/gifs/img_{img_idx}.gif", mode='I') as writer:
            for epoch in range(epochs):
                writer.append_data(imageio.imread(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png"))
                
    # delete the images
    for img_idx in range(4):
        for epoch in range(epochs):
            os.remove(f"{output_folder}/images/epoch_{epoch}_img_{img_idx}.png")
    os.rmdir(f"{output_folder}/images")


| ![](outputs/1/roi_head_outputs/gifs/img_0.gif) | ![](outputs/1/roi_head_outputs/gifs/img_1.gif) |
|---------|---------|
| ![](outputs/1/roi_head_outputs/gifs/img_2.gif) | ![](outputs/1/roi_head_outputs/gifs/img_3.gif) |
